# 🕵️ Forensische Analyse von Weblogs mit DuckLake und DuckDB## Einführung: DuckLake als Unveränderliches Archiv (Immutable Ledger)Dieses Notebook demonstriert die Kernfunktionen der **DuckLake-Erweiterung** für DuckDB, indem es eine forensische Untersuchung von Weblog-Daten durchführt. DuckLake speichert jeden Zustand als **versionierten Snapshot**, was die Rekonstruktion des Originalzustands (Time Travel) ermöglicht. Wir simulieren eine **Log-Manipulation** (Löschen von Fehlereinträgen) und nutzen die robusten DuckLake-Funktionen, um die Manipulation zu beweisen.

## 1. Vorbereitung und Daten-Setup

In [ ]:
import duckdbimport pandas as pdimport matplotlib.pyplot as pltimport osimport timefrom datetime import datetimeimport numpy as np
# Setze das Matplotlib Backend für Notebooks
%matplotlib inline
# Definiere den Tabellennamen global
table_name = 'weblogs'

In [ ]:
# 1. Konfiguriere die Pfade
DUCKLAKE_METADATA_PATH = 'ducklake_metadata.ducklake'
DUCKLAKE_DATA_PATH = 'ducklake_data_files'
CATALOG_NAME = 'weblog_lake'

# Erstelle die Verzeichnisse, falls sie noch nicht existieren
os.makedirs(DUCKLAKE_DATA_PATH, exist_ok=True)

# 2. Verbinde mit DuckDB
con = duckdb.connect(database='weblog_demo.duckdb')

# 3. Installiere und lade die DuckLake-Erweiterung
con.sql("INSTALL ducklake;")
con.sql("LOAD ducklake;")

# 4. Attache den DuckLake-Katalog
attach_query = f"""
    ATTACH 'ducklake:{DUCKLAKE_METADATA_PATH}' AS {CATALOG_NAME} (DATA_PATH '{DUCKLAKE_DATA_PATH}');
"""
con.sql(attach_query)

# 5. Setze den DuckLake-Katalog als Standard
con.sql(f"USE {CATALOG_NAME};")

print(f"✅ DuckLake '{CATALOG_NAME}' erfolgreich erstellt und verbunden.")

✅ DuckLake 'weblog_lake' erfolgreich erstellt und verbunden.


## 2. Simulation der Manipulation: Originalzustand (Snapshot 1)

In [ ]:
# 1. Erstelle einen Dummy-DataFrame (Originalzustand)
data = {
    'ip': ['10.0.0.1'] * 10000 + ['203.0.113.1'] * 1000 + ['192.168.1.1'] * 500 + ['172.16.0.1'] * 50,
    'timestamp': pd.to_datetime('2025-09-01 10:00:00') + pd.to_timedelta(np.arange(11550), unit='s'),
    'url': ['/home', '/about', '/product'] * 3850,
    'status_code': [200] * 11000 + [404] * 400 + [500] * 150
}
df_original = pd.DataFrame(data)

# 2. Registriere den DataFrame im DuckLake-Katalog (Dies erstellt Snapshot 1)
con.register("df_original_temp", df_original)
con.sql(f"CREATE OR REPLACE TABLE {table_name} AS SELECT * FROM df_original_temp;")
con.unregister("df_original_temp")

# 3. Hole die ID des ersten Snapshots
snapshot_1_id = con.execute(f"SELECT MAX(snapshot_id) FROM ducklake_snapshots('{CATALOG_NAME}')").fetchone()[0]

print(f"✅ Snapshot 1 (Originalzustand) erstellt mit ID: {snapshot_1_id}")

## 3. Simulation der Manipulation: Löschen und Hinzufügen von Einträgen (Snapshot 2)**Ziel der Manipulation:** Alle Statuscodes `404` und `500` löschen, um die Logs sauber erscheinen zu lassen, und einen einzelnen harmlosen Eintrag hinzufügen (Ablenkung).

In [ ]:
# 1. Lösche die kritischen Fehlercodes (Vertuschung)
con.sql(f"DELETE FROM {table_name} WHERE status_code >= 400;")

# 2. Füge einen harmlosen Eintrag hinzu (Ablenkung)
con.sql(f"INSERT INTO {table_name} VALUES ('203.0.113.99', '{pd.to_datetime('2025-09-01 11:00:00')}', '/robots.txt', 200);")

# 3. Hole die ID des zweiten Snapshots (neuester Zustand)
snapshot_2_id = con.execute(f"SELECT MAX(snapshot_id) FROM ducklake_snapshots('{CATALOG_NAME}')").fetchone()[0]

print(f"✅ Snapshot 2 (Manipulierter Zustand) erstellt mit ID: {snapshot_2_id}")

## 4. Forensische Analyse: Beweissicherung mit DuckLake-FunktionenDa die direkte Time-Travel-Syntax (z.B. `AS OF 1`) in dieser Umgebung fehlschlägt, nutzen wir die **explizit registrierten Tabellenfunktionen** von DuckLake, die den SQL-Parser umgehen und die Beweise direkt aus den Metadaten extrahieren.

In [ ]:
# Die vier Hauptfunktionen der forensischen Analyse:

# 1. ducklake_snapshots: Zeigt die gesamte Versionshistorie
print("\n[Beweis 1: Manipulationshistorie]\n")
con.sql(f"""
    SELECT 
        snapshot_id, 
        strftime(snapshot_time, '%Y-%m-%d %H:%M:%S') AS time,
        changes
    FROM 
        ducklake_snapshots('{CATALOG_NAME}')
    ORDER BY 
        snapshot_id;
""").show()
print("Die Snapshots zeigen exakt den Zeitpunkt des Originalzustands (1) und des manipulierten Zustands (2).")

# 2. ducklake_table_deletions: Zeigt die Vertuschung
print("\n[Beweis 2: Gelöschte Einträge (Vertuschung) zwischen Snapshot 1 und 2]\n")
con.sql(f"""
    SELECT 
        ip, 
        url,
        status_code 
    FROM 
        ducklake_table_deletions('{CATALOG_NAME}', 'main', '{table_name}', {snapshot_1_id}, {snapshot_2_id});
""").show()
print("Die Deletions-Funktion rekonstruiert exakt die von der Manipulation entfernten Beweismittel (alle Statuscodes >= 400).")

# 3. ducklake_table_insertions: Zeigt die Ablenkung
print("\n[Beweis 3: Neu hinzugefügte Einträge (Ablenkung) zwischen Snapshot 1 und 2]\n")
con.sql(f"""
    SELECT 
        ip, 
        url,
        status_code 
    FROM 
        ducklake_table_insertions('{CATALOG_NAME}', 'main', '{table_name}', {snapshot_1_id}, {snapshot_2_id});
""").show()
print("Die Insertions-Funktion zeigt den einzelnen, unverdächtigen Eintrag, der zur Verschleierung der Manipulation hinzugefügt wurde.")


[Beweis 1: Manipulationshistorie]

┌─────────────┬─────────────────────┬──────────────────────────┐
│ snapshot_id │         time        │         changes          │
│    int64    │        varchar      │ map(varchar, varchar[])  │
├─────────────┼─────────────────────┼──────────────────────────┤
│           1 │ 2025-09-30 14:10:17 │ {schemas_created=[main]} │
│           2 │ 2025-09-30 14:10:17 │ {data_files_deleted=[]}  │
└─────────────┴─────────────────────┴──────────────────────────┘
Die Snapshots zeigen exakt den Zeitpunkt des Originalzustands (1) und des manipulierten Zustands (2).

[Beweis 2: Gelöschte Einträge (Vertuschung) zwischen Snapshot 1 und 2]

┌───────────────┬─────────────────────┬─────────────┐
│      ip       │         url         │ status_code │
│    varchar    │       varchar       │    int64    │
├───────────────┼─────────────────────┼─────────────┤
│ 172.16.0.1    │ /home               │         404 │
│ 172.16.0.1    │ /about              │         500 │
│ 203.0.11

## 5. Forensische Statuscode-Analyse: Beweis der DatenmanipulationDieser Code vergleicht die **HTTP-Statuscode-Verteilung** des **aktuellen (manipulierten) Zustands** (Snapshot 2) mit den **gelöschten Einträgen** (die den Beweis der Manipulation darstellen). Da die direkte Time-Travel-Syntax fehlschlägt, wird die `ducklake_table_deletions`-Funktion genutzt, um die forensischen Beweise direkt abzurufen.

In [ ]:
# --- 1. Daten für den "Original (Beweis)" Zustand abrufen ---

# Die gelöschten Zeilen (deletions) entsprechen den Fehler-Einträgen, 
# die in der Original-Logdatei enthalten waren. Wir extrahieren sie per Funktion.
df_deletions = con.execute(f"""
    SELECT 
        CAST(status_code AS VARCHAR) AS status_code
    FROM 
        ducklake_table_deletions('{CATALOG_NAME}', 'main', '{table_name}', {snapshot_1_id}, {snapshot_2_id});
""").fetchdf()

# Wir führen die Analyse der gelöschten Einträge durch (die kritisch sind)
original_errors = df_deletions.groupby('status_code').size().reset_index(name='cnt')
original_errors['state'] = 'Original (Gelöschte Fehler)'
original_errors['version_id'] = 1


# --- 2. Daten für den "Aktuellen (Manipulierten)" Zustand abrufen ---
df_manipulated = con.execute(f"""
    SELECT 
        CAST(status_code AS VARCHAR) AS status_code,
        COUNT(*) AS cnt,
        2 AS version_id,
        'Manipuliert (Aktuell)' AS state
    FROM 
        {table_name}
    GROUP BY 
        status_code
""").fetchdf()


# --- 3. Visualisierung der relevanten Daten (Manipulation vs. Gelöschte Fehler) ---

# Kombiniere die Fehler aus dem Originalzustand und den aktuellen Zustand
combined_analysis = pd.concat([original_errors, df_manipulated], ignore_index=True)

# Funktion zur Farbbestimmung
def get_colors(df):
    colors = []
    for s in df['status_code']:
        try:
            if int(s) >= 400:
                colors.append('darkred')
            else:
                colors.append('green')
        except:
            colors.append('gray')
    return colors

df_original_errors = combined_analysis[combined_analysis['version_id'] == 1].copy()

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Manipulierter Zustand (Aktuell)
df_manipulated.plot.bar(x="status_code", y="cnt", legend=False, ax=axes[0], color=get_colors(df_manipulated))
axes[0].set_title("HTTP-Statuscode Verteilung (Aktuell - Manipuliert)")
axes[0].set_ylabel("Anzahl der Anfragen")
axes[0].set_xlabel("Statuscode")
axes[0].tick_params(axis='x', rotation=0)

# Plot 2: Historischer Beweis (Nur die gelöschten Fehlercodes)
df_original_errors.plot.bar(x="status_code", y="cnt", legend=False, ax=axes[1], color='darkred')
axes[1].set_title("HTTP-Statuscode Verteilung (Forensischer Beweis: Gelöschte Fehler)")
axes[1].set_ylabel("Anzahl der Anfragen")
axes[1].set_xlabel("Statuscode")
axes[1].tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.show()

print("\nStatuscode-Analyse der gelöschten Einträge (Forensischer Beweis):")
print(df_original_errors)


Statuscode-Analyse der gelöschten Einträge (Forensischer Beweis):
  status_code  cnt                      state  version_id
0         404  400  Original (Gelöschte Fehler)           1
1         500  150  Original (Gelöschte Fehler)           1


## 6. Abschluss und BeweissicherungDie robusten **DuckLake-Tabellenfunktionen** haben es ermöglicht, **unveränderliche Beweise** (gelöschte Fehler-Logs, Statuscode-Anomalie) aus dem historischen Log zu extrahieren. Das Archiv ist nun gesichert und die Manipulation belegt.

In [ ]:
# Schließe die DuckDB-Verbindung und speichere alle Metadaten final ab
con.close()
print("✅ Forensische Analyse abgeschlossen. Das Archiv ist gesichert.")